# 飲食店のタスク割当問題

多くの従業員を擁する小売業やサービス業では、日々変動する多種多様な作業に対して、各従業員の役職やスキルを考慮して業務(タスク)を割り当てることが重要です。

ここでは、飲食チェーン店を例として、従業員を店舗に割り当てる組合せ最適化問題に取り組みます。  
適切な割り当てを行う事により、各店舗の業務に滞り無いように、業務の全体的な効率化と業務量の平準化を目指します。

各従業員には次のような属性を持つこととします。

*   (役職) 店長・副店長・食品衛生管理者・なし
*   (役割) 調理担当・ホール担当
*   (スキル) 調理スキル (複数)
*   (店舗) 勤務可能場所

また、各店舗には下記の要求(制約条件)が課せられているとします。

*   役職、役割ごとの必要人数
*   必要な調理スキル量

以上を考慮した上で、各店舗、各役割に対する要求従業員数の充足率の最大化と分散の最小化を設定し、店舗毎の従業員割り当ての効率化と偏りの平準化を狙います。

一度に全ての条件を考慮するのは複雑なので、少しずつ条件を増やしながら定式化を行います。

## ステップ1: 勤務店舗の割当

まずは、役職や役割を考慮せずに、従業員の店舗への割り当てのみを考えます。  
各店舗には必要な従業員の人数が設定されており、一方で各従業員には

*   0: 勤務不可能
*   1: 勤務可能
*   2: 勤務希望

という値が店舗毎に設定されていることにします。  
そこで、店舗の要求人数満たしつつ、従業員の希望度に剃った沿った勤務地の割当を考えます。

### 定式化

最初に定式化で用いる変数と記号を定義します。

#### 集合

*   $W$: 従業員集合 (記号 $i$ を用いて従業員を表す)
*   $S$: 店舗集合 (記号 $l$ を用いて店舗を表す)

#### 定数

*   $r_{l}$: 店舗 $l \in S$ の要求従業員数
*   $c_{i,l}$: 従業員 $i \in W$ が店舗 $l \in S$ で勤務する場合の希望度

#### 決定変数

*   $L_{i,l}\in \{0,1\}$: 従業員 $i\in W$ を店舗  $l\in S$ に割当てるか

最適化の結果として出力される情報はどの従業員がどの店舗に割り当てられるのかを表します。したがって、従業員数 × 店舗数の変数 $L$ を $\{0,1\}$ 変数として宣言します。 

例えば、従業員数が5名で、店舗として「博多店」と「天神店」がある場合、2 x 5 の10変数で表現されます。

---
| 従業員id|$博多$|$天神$|
|:---:|:---:|:---:|
| $0$|$L_{0,0}$|$L_{0,1}$
| $1$|$L_{1,0}$|$L_{1,1}$
| $2$|$L_{2,0}$|$L_{2,1}$
| $3$|$L_{3,0}$|$L_{3,1}$
| $4$|$L_{4,0}$|$L_{4,1}$
---

#### 目的関数

店舗に従業員の偏りが起こることは望ましくありません。そこで、各店舗の要求従業員数に対する割当従業員の充足率を定義し下記を目的関数とします。

* 充足率の平均最大化
* 充足率の分散最小化
* 従業員の勤務希望地の最大化

**充足率の定義**

店舗$l$における要求人数に対する従業員数の充足率$w_l$

$$
\begin{align*}
    w_l := \frac{1}{r_l} \displaystyle\sum_{i\in W} L_{i,l}
\end{align*}
$$
    
**充足率の平均の最大化**

$$
\begin{matrix}
    {\rm maximize} & \left< w_l \right>
\end{matrix}
$$



**充足率の分散の最小化**

$$
\begin{matrix}
    {\rm minimize} & \left< w_l^2 \right> - \left< w_l \right>^2
\end{matrix}
$$

**従業員の希望度の最大化**

$$
\begin{matrix}
    {\rm maximize}& \displaystyle \sum_{i\in W}\sum_{l\in S}c_{i,l}L_{i,l}
\end{matrix}
$$
    
#### 制約条件

先ほどの決定変数の定義では、同じ従業員が同時に複数の店舗に勤務するような状況が発生するため、それを明示的に禁止するような制約を与えます。また、店舗毎に要求人数が定数として設定されています。

**従業員$i$は同時に1店舗のみ勤務する**

$$
\begin{align*}
    \forall i\in W, \sum_{l \in S}L_{i,l} = 1
\end{align*}
$$

**各店舗の要求人数以上の人員を配置する**

$$
\begin{align*}
    \forall l \in S, \sum_{i\in W}L_{i,l} \geq r_{l}
\end{align*}
$$

### データの作成

上記の定式化において、各店舗の要求人数の情報と各従業員の勤務希望情報が入力として与えられます。例として、従業員数5名、店舗数2のデータを下記の様に作成します。  
データの格納には`pandas.DataFrame`を使用します。


In [ ]:
import pandas as pd

# 各店舗の要求人数情報の読み込み
dict_req = dict(location=["tenjin", "hakata"], employee=[2, 3])

# 各従業員の勤務希望情報の読み込み
dict_worker_loc = dict(
    worker_id=[0, 1, 2, 3, 4], tenjin=[2, 2, 1, 0, 1], hakata=[1, 1, 1, 1, 0]
)

df_req = pd.DataFrame.from_dict(dict_req, orient="index").T
print("各店舗の従業員要求人数")
display(df_req.style.hide_index())
df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T
print("\n各従業員の勤務希望情報")
display(df_worker_loc.style.hide_index())

## 店舗名の取得
workers = df_worker_loc["worker_id"].values
locations = df_req["location"].values

## 各データ長を取得
num_workers = len(workers)
num_locations = len(locations)

店舗名と変数のインデックスをそれぞれkeyとvalueとする辞書を作成しておきます。

In [ ]:
# dictの作成
idx2loc = dict((i, v) for i, v in enumerate(df_req["location"].values))
loc2idx = dict((v, i) for i, v in enumerate(df_req["location"].values))

### Amplify による定式化

それでは Amplify を用いて定式化します。最初に変数 $L$ を表す変数 `location_variables` を `BinarySymbolGenerator` クラスで作成します。

In [ ]:
# 従業員iが勤務地lで勤務することを表す変数
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()
location_variables = gen.array(num_workers, num_locations)

勤務不可能店舗に関しては勤務しないことがわかっているため、`location_variables` に先に定数を与えます。これは問題の簡略化に役立ちます。変数を定数化する場合は `BinaryPoly` クラスの定数として与えます。

In [ ]:
## 勤務不可能地域に関しては変数を定数化
from itertools import product

for i, l in product(range(num_workers), locations):
    worker_req = df_worker_loc.iloc[i][l]
    if worker_req == 0:
        # 勤務不可
        location_variables[i, loc2idx[l]] = 0

## 勤務不可能地域に関する変数が0に固定されている
print(location_variables)

次に各店舗の要求人数に対する充足率 `w` を計算します。

$$
\begin{align*}
    w_l := \frac{1}{r_l} \displaystyle\sum_{i\in W} L_{i,l}
\end{align*}
$$

In [ ]:
## 充足率の計算
from amplify import sum_poly, BinaryPolyArray

w = location_variables.sum(axis=0) / df_req["employee"].values

そして目的関数の各項目を計算します。最大化する目的についてはマイナスを付けることで最小化問題に変換します。

**充足率の平均の最大化**

$$
\begin{matrix}
    {\rm minimize} & -\left< w_l \right>
\end{matrix}
$$

**充足率の分散の最小化**

$$
\begin{matrix}
    {\rm minimize} & \left< w_l^2 \right> - \left< w_l \right>^2
\end{matrix}
$$

**従業員の希望度の最大化**

$$
\begin{matrix}
    {\rm minimize}& \displaystyle -\sum_{i\in W}\sum_{l\in S}c_{i,l}L_{i,l}
\end{matrix}
$$

In [ ]:
# 充足率の平均の最大化
average_fill_rate_cost = -((w.sum() / w.size) ** 2)

# 充足率の分散の最小化
variance_fill_rate_cost = (w * w).sum() / w.size - (w.sum() / w.size) ** 2

# 従業員の希望度最大化
location_cost = -sum_poly(
    num_workers,
    lambda i: sum_poly(
        num_locations,
        lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i, l],
    ),
)

最後に制約条件を与えます。

**従業員$i$は同時に1店舗のみ勤務する**

$$
\begin{align*}
    \forall i\in W, \sum_{l \in S}L_{i,l} = 1
\end{align*}
$$

**各店舗要求人数以上の人員を配置する**

$$
\begin{align*}
    \forall l \in S, \sum_{i\in W}L_{i,l} \geq r_{l}
\end{align*}
$$

一つ目の制約条件はOne-hot制約なので、`one_hot` を用いて記述します。(あるいは等式制約なので `equal_to` を用いて記述することもできます。)
二つ目の制約式は不等式制約なので `greater_equal` を用いて記述します。

In [ ]:
from amplify.constraint import equal_to, greater_equal, one_hot

# 従業員iは同時に1店舗のみ勤務できる
location_constarints = sum(one_hot(location_variables[i]) for i in range(num_workers))

# 店舗の合計人数は要求人数以上
require_constraints = sum(
    greater_equal(location_variables[:, l], df_req["employee"][l])
    for l in range(num_locations)
)
# require_constraints = greater_equal(location_variables.sum(axis=0), df_req["employee"].values).sum()

以上の目的関数と制約式から最適化モデルを作成します。  

目的関数はそれぞれの項目に係数をかけて優先順位を設定します。下記の `loc_priority`, `ave_fill_priority`, `var_fill_priority` はそれぞれ、勤務地希望度、充足率平均、充足率分散に対応する係数です。この時、それぞれの目的関数がどの程度の値になるのかを考慮して係数を決定します。例えば分散は正の小さい値になるので、目的関数として考慮させるには大きな係数を与える必要があるでしょう。

In [ ]:
# それぞれの目的関数の係数
loc_priority = 1
ave_fill_priority = 1
var_fill_priority = 10

# 目的関数
cost_func = (
    loc_priority * location_cost
    + ave_fill_priority * average_fill_rate_cost
    + var_fill_priority * variance_fill_rate_cost
)

制約条件についても適切に強さを設定する必要があります。制約条件は目的関数に対するペナルティ関数としてイジングマシンに与えられるため、目的関数の取り得る値より大きめの値を推定して決定します。ここでは $10$ 程度の値を与えておけば十分でしょう。

In [ ]:
# 制約条件を表すペナルティ関数の重み
constraint_weight = 10

# 制約条件
constraints = constraint_weight * (location_constarints + require_constraints)

以上で定式化は完了です。次に使用するイジングマシンのクライアントを設定します。

In [ ]:
from amplify.client import FixstarsClient

# クライアントの設定
client = FixstarsClient()
# client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # ローカル環境では Amplify AEのアクセストークンを入力してください
client.parameters.timeout = 1000  #  タイムアウト１秒

クライアントからソルバーを作成し、定式化したモデルを最適化します。

In [ ]:
from amplify import Solver

# 最適化モデル
model = cost_func + constraints

# ソルバーを定義して実行
solver = Solver(client)
result = solver.solve(model)

変数 `location_variables` に対応した解を取得するために `decode` メソッドを用いて解を取り出します。取得した解を `location_solutions` とします。

In [ ]:
# 制約条件チェック
if len(result) == 0:
    raise RuntimeError("The given constraints are not satisfied")
values = result[0].values
energy = result[0].energy

# 勤務地に関する変数の解
location_solutions = location_variables.decode(values, 0)

### 結果の解析

結果から従業員がそれぞれがどこの店舗で勤務を行うのか出力します。 変数 `location_solutions`において、`location_solutions[i][l]` が $1$ であれば従業員 $i$ は店舗 $l$ で勤務することを表します。したがって、解が$1$であるインデックスを取り出すことで、どの従業員がどの店舗で勤務するのかを取得できます。

表データで出力するために、`pandas.DataFrame`に結果を格納します。

In [ ]:
import numpy as np
from collections import defaultdict

location_index_list = np.where(np.array(location_solutions) == 1)[1]
dict_df = defaultdict(list)

for i, loc_ind in enumerate(location_index_list):
    ## 配属勤務地
    worker_id = df_worker_loc.loc[i]["worker_id"]
    loc = locations[loc_ind]
    dict_df["worker_id"].append(worker_id)
    dict_df["location"].append(loc)

df_result = pd.DataFrame.from_dict(dict_df, orient="index").T
display(df_result.style.hide_index())

最後にどの程度店舗の要求人数を満たしているか確認します。次のようにして、店舗の割り当て要求人数を合計人数で割ることで得られます。

In [ ]:
num_member = df_result["location"].value_counts()
fill_rate = df_req.copy()
fill_rate["fill_rate"] = [
    df_req.loc[l]["employee"] / num_member[idx2loc[l]] for l in range(num_locations)
]
display(fill_rate.style.hide_index())

以上より、従業員割り当てが上手く行われたことが確認できました。

## ステップ2: 店舗と役職の割当

ステップ1の設定条件に加えて、各店舗の**役職毎**の要求人数を満たすような従業員割り当てを行います。具体的には、ある店舗Aでは店長が1人、一般従業員３人が必要なので、それを満たすように人員を配置するといった問題です。例として、「店長・副店長・一般従業員」という役職を用意します。ただし、店長と副店長は兼任できないものとします。各従業員には先ほどの勤務希望地に加えて、それぞれの役職が担当可能かどうかを表す属性を持つことにします。


![img](../figures/employee_step2.png "ex: コンビニの従業員割り当て")

### 定式化

定式化に用いる変数と記号を定義し直します。

#### 集合

*   $W$: 従業員集合 (記号 $i$ を用いて従業員を表す)
*   $S$: 店舗集合 (記号 $l$ を用いて店舗を表す)
*   $R$: 役職集合 (記号 $j$ を用いて役職を表す)
    *   $j=0$: 店長
    *   $j=1$: 副店長
    *   $j=2$: なし 

#### 定数

*   $r_{j,l}$: 店舗 $l \in S$ の役職 $j \in R$ の要求従業員数
*   $c_{i,l}$: 従業員 $i \in W$ が店舗 $l \in S$ で勤務する場合の希望度
*   $m_{i,j}$: 従業員 $i \in W$ が役職 $j \in R$ に割当て可能か (0:不可能, 1:可能)

#### 決定変数

*   $M_{i,j,l}\in \{0,1\}$: 従業員 $i\in W$ を店舗 $l\in S$ で役職 $j\in R$ へ割当てるか
*   $L_{i,l}\in \{0,1\}$: 従業員 $i\in W$ を店舗  $l\in S$ に割当てるか
    *   $L_{i,l} = \sum_{j \in R} M_{i,j,l}$ の関係がある

#### 目的関数

ステップ1と同様に、各店舗の役職に対する要求従業員数に対する割当従業員の充足率を定義し、下記を目的関数とします。

* 充足率の平均最大化
* 充足率の分散最小化
* 従業員の勤務希望地の最大化

**充足率の定義**

店舗$l$における要求人数に対する従業員数の充足率$w_l$

$$
\begin{align*}
    w_l := \frac{\displaystyle \sum_{i\in W} L_{i,l}}{\displaystyle \sum_{j\in R} r_{j,l}}  
\end{align*}
$$
    
**充足率の平均の最大化**

$$
\begin{matrix}
    {\rm maximize} & \left< w_l \right>
\end{matrix}
$$

**充足率の分散の最小化**

$$
\begin{matrix}
    {\rm minimize} & \left< w_l^2 \right> - \left< w_l \right>^2
\end{matrix}
$$

**従業員の希望度の最大化**

$$
\begin{matrix}
    {\rm maximize}& \displaystyle \sum_{i\in W}\sum_{l\in S}c_{i,l}L_{i,l}
\end{matrix}
$$
    
#### 制約条件

ステップ1に役職に関する制約を加えます。

**従業員$i$は同時に1店舗のみ勤務する**

$$
\begin{align*}
    \forall i\in W, \sum_{l \in S}L_{i,l} = 1
\end{align*}
$$

**各店舗の要求人数に等しい管理職を配置する**

$$
    \forall l \in S, \forall j \in \left\{0, 1 \right\}, \sum_{i\in W}M_{i,j,l} = r_{j,l}
$$

**各店舗の要求人数以上の従業員を配置する**

$$
\begin{align*}
    \forall l \in S, \sum_{i\in W} L_{i,l} \geq \sum_{j\in R}r_{j,l}
\end{align*}
$$

### データの作成

上記の定式化において、各店舗の要求人数の情報と各従業員の勤務希望とスキル情報が入力として与えられます。例として、従業員数9名、店舗数4のデータを下記に作成します。  

In [ ]:
import pandas as pd

# 各店舗の要求役割情報
dict_req = dict(
    location=["tenjin", "hakata", "akasaka", "gakken"],
    manager=[1, 1, 1, 1],
    submanager=[1, 0, 1, 1],
    employee=[2, 2, 2, 2],
)
df_req = pd.DataFrame.from_dict(dict_req, orient="index").T

# 各従業員の勤務希望情報
dict_worker_loc = dict(
    worker_id=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    tenjin=[2, 0, 0, 0, 1, 1, 2, 1, 1],
    hakata=[1, 0, 0, 2, 2, 2, 1, 2, 1],
    akasaka=[1, 0, 0, 1, 0, 1, 1, 1, 2],
    gakken=[1, 2, 2, 0, 0, 0, 0, 0, 0],
)
df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T

# 各従業員のスキル情報の読み込み
dict_worker_skill = dict(
    worker_id=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    manager=[1, 1, 0, 0, 1, 1, 1, 0, 1],
    submanager=[1, 1, 1, 0, 1, 1, 1, 0, 1],
    employee=[1, 1, 1, 1, 1, 1, 1, 1, 1],
)
df_worker_skill = pd.DataFrame.from_dict(dict_worker_skill, orient="index").T

各店舗の manager(店長), submanager(副店長), employee(全従業員)に関する要求人数が `df_req` に格納されています。簡単のために、`df_req["employee"]` には $\sum_{j\in R} r_{j,l}$ を与えます。

In [ ]:
display(df_req.style.hide_index())

各従業員の各店舗に対する勤務希望度は `df_worker_loc` に格納されています。

In [ ]:
display(df_worker_loc.style.hide_index())

`df_worker_skill` には、各従業員の役職情報を格納しています。もし値が $1$ ならその役職が担当可能であることを表します。
例えば、`worker_id = 1` の従業員は店長と副店長が担当可能であることがわかります。一方 `worker_id = 7` の従業員は管理職を担当することができません。

In [ ]:
display(df_worker_skill.style.hide_index())

次のように従業員id、店舗名、役職名とインデックスの対応関係を調べておきます。

In [ ]:
# 従業員id
workers = df_worker_loc["worker_id"].values
# 店舗名
locations = df_req["location"].values
# 役職名
roles = ["manager", "submanager", "employee"]

# dictの作成
idx2loc = dict((i, v) for i, v in enumerate(locations))
loc2idx = dict((v, i) for i, v in enumerate(locations))
idx2role = dict((i, v) for i, v in enumerate(roles))
role2idx = dict((v, i) for i, v in enumerate(roles))

# 各データ長を取得
num_workers = len(workers)
num_locations = len(locations)
num_roles = len(roles)

### Amplify による定式化

それでは Amplify を用いて定式化します。最初に変数 $M$ を表す変数 `role_variables` を `BinarySymbolGenerator` で作成します。従業員、役職、店舗の3次元配列であることに注意してください。

In [ ]:
# 従業員iが役職jで店舗lに勤務することを表す変数
role_variables = BinarySymbolGenerator().array(num_workers, num_roles, num_locations)

ステップ1と同様に勤務できない店舗や役職に関しては事前に変数の値を代入しておきます。

In [ ]:
for i, l in product(range(num_workers), locations):
    worker_req = df_worker_loc.iloc[i][l]
    if worker_req == 0:
        # 全ての役職で店舗割当が不可
        role_variables[i, :, loc2idx[l]] = 0

for i, j in product(range(num_workers), roles):
    worker_skill = df_worker_skill.iloc[i][j]
    if worker_skill == 0:
        # 全ての店舗で役職が不可
        role_variables[i, role2idx[j], :] = 0

## 勤務不可能店舗・役職に関する変数が0に固定されている
print(role_variables)

変数 $L$ を表す `location_variables` については、変数 $M$ から次のようにして得られます。

In [ ]:
from amplify import BinaryPolyArray

location_variables = role_variables.sum(axis=1)
display(location_variables)

次に各店舗の要求人数に対する充足率 $w$ を計算します。これはステップ1と同様です。

$$
\begin{align*}
    w_l := \frac{\displaystyle \sum_{i\in W} L_{i,l}}{\displaystyle \sum_{j\in R} r_{j,l}}  
\end{align*}
$$

分母の $\sum_{j\in R} r_{j,l}$ は計算済みで `df_req["employee"]` で与えられることに注意してください。

In [ ]:
# 充足率の計算
w = location_variables.sum(axis=0) / df_req["employee"].values
display(w)

目的関数の各項目を計算します。最大化する目的についてはマイナスを付けることで最小化問題に変換します。

**充足率の平均の最大化**

$$
\begin{matrix}
    {\rm minimize} & -\left< w_l \right>
\end{matrix}
$$

**充足率の分散の最小化**

$$
\begin{matrix}
    {\rm minimize} & \left< w_l^2 \right> - \left< w_l \right>^2
\end{matrix}
$$

**従業員の希望度の最大化**

$$
\begin{matrix}
    {\rm minimize}& \displaystyle -\sum_{i\in W}\sum_{l\in S}c_{i,l}L_{i,l}
\end{matrix}
$$

In [ ]:
# 充足率の平均の最大化
average_fill_rate_cost = -((w.sum() / w.size) ** 2)

# 充足率の分散の最小化
variance_fill_rate_cost = (w * w).sum() / w.size - (w.sum() / w.size) ** 2

# 従業員の希望度最大化
location_cost = -sum_poly(
    num_workers,
    lambda i: sum_poly(
        num_locations,
        lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i][l],
    ),
)

In [ ]:
df_worker_loc

最後に制約条件を与えます。

**従業員$i$は同時に1店舗のみ勤務する**

$$
\begin{align*}
    \forall i\in W, \sum_{l \in S}L_{i,l} = 1
\end{align*}
$$

**各店舗の要求人数に等しい管理職を配置する**

$$
    \forall l \in S, \forall j \in \left\{0, 1 \right\}, \sum_{i\in W}M_{i,j,l} = r_{j,l}
$$

**各店舗の要求人数以上の従業員を配置する**

$$
\begin{align*}
    \forall l \in S, \sum_{i\in W} L_{i,l} \geq \sum_{j\in R} r_{j,l}
\end{align*}
$$

一つ目と二つ目の制約条件は、等式制約なので `equal_to` を用いて記述します。三つ目の制約式は不等式制約なので `greater_equal` を用いて記述します。


In [ ]:
# 従業員iは同時に1店舗のみ勤務できる
location_constarints = sum(one_hot(location_variables[i]) for i in range(num_workers))

# 各店舗の要求人数に等しい管理職を配置する
req_manager_constraints = sum(
    equal_to(role_variables[:, 0, l], df_req["manager"][l])
    for l in range(num_locations)
)
req_submanager_constraints = sum(
    equal_to(role_variables[:, 1, l], df_req["submanager"][l])
    for l in range(num_locations)
)

# 各店舗の要求人数以上の従業員を配置する
req_employee_constraints = sum(
    greater_equal(location_variables[:, l], df_req["employee"][l])
    for l in range(num_locations)
)

目的関数と制約式から最適化モデルを作成します。ステップ1と同様に適切な重みを与えます。制約条件の重みは目的関数の取り得る値より大きめの値を推定して決定します。ここでは $20$ 程度の値を与えておけば十分でしょう

In [ ]:
# それぞれの目的関数の係数
loc_priority = 1
ave_fill_priority = 1
var_fill_priority = 10

# 目的関数
cost_func = (
    loc_priority * location_cost
    + ave_fill_priority * average_fill_rate_cost
    + var_fill_priority * variance_fill_rate_cost
)

# 制約条件を表すペナルティ関数の重み
constraint_weight = 20

# 制約条件
constraints = constraint_weight * (
    location_constarints
    + req_manager_constraints
    + req_submanager_constraints
    + req_employee_constraints
)

# 最適化モデル
model = cost_func + constraints

イジングマシンを実行します

In [ ]:
# 最適化の実行
result = solver.solve(model)

# 制約条件チェック
if len(result) == 0:
    raise RuntimeError("The given constraints are not satisfied")
values = result[0].values
energy = result[0].energy

`decode` メソッドを用いて、変数 `location_varibales`, `role_variables` の解として取り出します。取得した解をそれぞれ `location_solutions`, `role_solutions` とします。

In [ ]:
# 割当店舗に関する変数の解
location_solutions = location_variables.decode(values)

# 割当店舗と役職に関する変数の解
role_solutions = role_variables.decode(values)

### 結果の解析

結果から従業員がそれぞれがどこの店舗で勤務を行うのかを出力します。 変数 `role_solutions` において、`role_solutions[i][j][l] = 1` であれば、従業員 $i$ は役職 $j$ として店舗 $l$ で勤務することを表します。したがって、解が$1$であるインデックスを取り出すことで、どの従業員がどの役職、店舗で勤務するのかを取得できます。



In [ ]:
import numpy as np
from collections import defaultdict

(role_index_list, loc_index_list) = np.where(np.array(role_solutions) == 1)[1:]
dict_df = defaultdict(list)

for i, (j, l) in enumerate(zip(role_index_list, loc_index_list)):
    ## 配属勤務地
    worker_id = df_worker_loc.loc[i]["worker_id"]
    role = roles[j]
    loc = locations[l]
    dict_df["worker_id"].append(worker_id)
    dict_df["role"].append(role)
    dict_df["location"].append(loc)

df_result = pd.DataFrame.from_dict(dict_df, orient="index").T
display(df_result.style.hide_index())

つぎに、各店舗の役割毎に、要求人数を満たしているか確認します。各役割に対する充足率 (fill rate) を出力します。ここで、表のセルの成分が`None`となっている箇所は、その役割の要求人数が0人であることを表します。

In [ ]:
dict_result_loc = defaultdict(lambda: defaultdict(int))
for loc, role in product(locations, roles):
    dict_result_loc[loc][role] = 0

for i in range(len(df_result)):
    data = df_result.loc[i]
    role = data["role"]
    location = data["location"]
    dict_result_loc[location][role] += 1

    if role != "employee":
        dict_result_loc[location]["employee"] += 1

df_result_loc = pd.DataFrame.from_dict(dict_result_loc, orient="index")
dict_result_loc = defaultdict(defaultdict)
for i in range(len(df_result_loc)):
    data = df_result_loc.iloc[i]
    loc = data.name
    for role in roles:
        num_req = df_req[df_req["location"] == loc][role].item()
        num_wariate = data[role].item()
        dict_result_loc[loc][
            f"{role}_fillrate"
        ] = f"{num_wariate/num_req if num_req > 0 else None}"

df_result_loc = pd.DataFrame.from_dict(dict_result_loc, orient="index")
display(df_result_loc.style)

各店舗要求役割人数を満たしており、博多のみ employee が１人多く割り当てられていることがわかります。

## ステップ3: 店舗と役職と役割の割当

ステップ2の設定条件に加えて、各従業員が担当する役割を細分化します。寿司チェーン店を想定して、各従業員は「捌き・握り・汁物・一品」といった調理スキルを持つことを想定し、各店舗の要求するスキル量を満たすように従業員を配置します。例えばある調理スキル値の要求量が10である店舗では、割り当て後の従業員の調理スキルの総和が10以上である必要があります。ステップ2までは店舗割当人数の割合を充足率としていましたが、ステップ3では各調理スキル要求に対する充足率を最適化します。

さらに、従業員には「ホール担当」「キッチン担当」のどちらかの役割を持たせます。ホールに割り当てる人数は与えられるものとして、調理スキル値が0の従業員はホール専任とします。

<img src="../figures/employee_step3_1.png">

|<img src="../figures/employee_step3_2.png">|<img src="../figures/employee_step3_3.png">|<img src="../figures/employee_step3_4.png">| <img src="../figures/employee_step3_5.png">|
| :---: | :---: | :---: | :---: |
| **捌き担当** | **握り担当** | **汁物担当** | **一品担当** | 

### 定式化

定式化に用いる変数と記号を定義し直します。

#### 集合

*   $W$: 従業員集合 (記号 $i$ を用いて従業員を表す)
*   $S$: 店舗集合 (記号 $l$ を用いて店舗を表す)
*   $R$: 役職集合 (記号 $j$ を用いて役職を表す)
    *   $j=0$: 店長
    *   $j=1$: 副店長
    *   $j=2$: なし
*   $K$: 調理スキル集合 (記号 $k$ を用いてスキルを表す)
    *   $k=0$: 捌き
    *   $k=1$: 握り
    *   $k=2$: 汁物
    *   $k=3$: 一品
*   $A$: 役割集合 (記号 $h$ を用いて役割を表す)
    *   $h=0$: ホール担当
    *   $h=1$: キッチン担当

#### 定数

*   $t_{k,l}$: 店舗 $l \in S$ の調理スキル $k \in K$ の要求スキル量
*   $r_{l}$: 店舗 $l \in S$ のホール担当の要求人数
*   $c_{i,l}$: 従業員 $i \in W$ が店舗 $l \in S$ で勤務する場合の希望度
*   $m_{i,j}$: 従業員 $i \in W$ が役職 $j \in R$ に割当て可能か (0:不可能, 1:可能)
*   $s_{i,k}$: 従業員 $i$ が役割 $k\in A$ を担当する場合のスキル量

#### 決定変数

*   $M_{i,j,l}\in \{0,1\}$: 従業員 $i\in W$ を店舗 $l\in S$ で役職 $j\in R$ へ割当てるか
*   $P_{i,h,l}\in \{0,1\}$: 従業員 $i\in W$ を店舗 $l\in S$ で役割 $h\in A$ へ割当てるか
*   $L_{i,l}\in \{0,1\}$: 従業員 $i\in W$ を店舗  $l\in S$ に割当てるか
    *   $L_{i,l} = \sum_{j \in R} M_{i,j,l}$ の関係がある
    *   $L_{i,l} = \sum_{h \in A} P_{i,h,l}$ の関係がある

#### 目的関数

ステップ1と同様に、各店舗の役職に対する要求従業員数に対する割当従業員の充足率を定義し、下記を目的関数とします。

* 充足率の平均最大化
* 充足率の分散最小化
* 従業員の勤務希望地の最大化

**充足率の定義**

店舗 $l$ における要求調理スキル $k$ に対する従業員数の充足率 $w_{k,l}$

$$
\begin{align*}
    w_{k,l} := \frac{1}{t_{k,l}}  \displaystyle \sum_{i\in W} s_{i,k} P_{i,1,l}
\end{align*}
$$
    
**充足率の平均の最大化**

$$
\begin{matrix}
    {\rm maximize} & \left< w_{k,l} \right>
\end{matrix}
$$

**充足率の分散の最小化**

$$
\begin{matrix}
    {\rm minimize} & \left< w_{k,l}^2 \right> - \left< w_{k,l} \right>^2
\end{matrix}
$$

**従業員の希望度の最大化**

$$
\begin{matrix}
    {\rm maximize}& \displaystyle \sum_{i\in W}\sum_{l\in S}c_{i,l}L_{i,l}
\end{matrix}
$$
    
#### 制約条件

基本的な考え方はステップ2と同様ですが、新しく定義した役割に関する変数 $P$ に関して $M$ との関係を制約する記述が必要です。

**従業員$i$は同時に1店舗のみ勤務する**

$$
\begin{align*}
    \forall i\in W, \sum_{l \in S}L_{i,l} = 1
\end{align*}
$$

**各店舗の要求人数に等しい管理職を配置する**

$$
    \forall l \in S, \forall j \in \left\{0, 1 \right\}, \sum_{i\in W}M_{i,j,l} = r_{j,l}
$$

**各店舗の要求人数に等しいホール担当を配置する**

$$
\begin{align*}
    \forall l \in S, \sum_{i\in W} P_{i,0,l} = r_{l}
\end{align*}
$$

**ある店舗で役職を持つ(持たない)ならその店舗で役割を持つ(持たない)**

$$
    \forall i\in W, \forall l \in S, \sum_{j \in R} M_{i,j,l} = \sum_{h \in A} P_{i,h,l}
$$

### データの作成

上記の定式化において、各店舗の要求スキル量・人数の情報と各従業員の勤務希望とスキル情報が入力として与えられます。例として、従業員数5名、店舗数2のデータを下記に作成します。  

In [ ]:
# 各店舗の要求スキル情報の読み込み
## store_require.csv
dict_req = dict(
    location=["tenjin", "hakata"],
    manager=[1, 1],
    submanager=[0, 1],
    sabaki=[1, 1],  # 捌き
    nigiri=[1, 2],  # 握り
    soup=[2, 2],  # 汁物
    ippin=[2, 2],  # 一品
    hall=[1, 1],  # ホール
)
df_req = pd.DataFrame.from_dict(dict_req, orient="index").T

# 各従業員の勤務希望情報の読み込み
dict_worker_loc = dict(
    worker_id=[0, 1, 2, 50, 43], tenjin=[2, 1, 1, 1, 1], hakata=[1, 2, 1, 1, 1]
)
df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T

# 各従業員のスキル情報の読み込み
dict_worker_skill = dict(
    worker_id=[0, 1, 2, 50, 43],
    manager=[1, 1, 0, 0, 0],
    submanager=[1, 1, 0, 1, 1],
    employee=[1, 1, 1, 1, 1],
    sabaki=[2, 2, 0, 1, 1],  # 捌き
    nigiri=[2, 2, 0, 2, 2],  # 握り
    soup=[2, 2, 0, 0, 0],  # 汁物
    ippin=[2, 2, 0, 1, 1],  # 一品
)
df_worker_skill = pd.DataFrame.from_dict(dict_worker_skill, orient="index").T

各店舗の manager(店長), submanager(副店長) に関する要求人数と、要求調理スキル量、ホール担当人数が `df_req` に格納されています。

In [ ]:
display(df_req.style.hide_index())

各従業員の各店舗に対する勤務希望度は df_worker_loc に格納されています。

In [ ]:
display(df_worker_loc.style.hide_index())

`df_worker_skill` には、各従業員のスキル情報を格納しています。`manager` と `submanager` については、もし値が $1$ ならその役職が担当可能であることを表します。一方、`sabaki`, `nigiri`, `soup`, `ippin` についてはそれぞれの調理スキルレベルを表します。$0$ の場合はキッチン担当ができない (ホール専任) ということを表します。

In [ ]:
display(df_worker_skill.style.hide_index())

次のように従業員id、店舗名、役職名とインデックスの対応関係を調べておきます。

In [ ]:
# 従業員id
workers = df_worker_loc["worker_id"].values
# 店舗名
locations = df_req["location"].values
# 役職名
roles = ["manager", "submanager", "employee"]
# 役割名
assigns = ["hall", "kitchen"]
# 調理スキル名
skills = ["sabaki", "nigiri", "soup", "ippin"]

# dictの作成
idx2loc = dict((i, v) for i, v in enumerate(locations))
loc2idx = dict((v, i) for i, v in enumerate(locations))
idx2role = dict((i, v) for i, v in enumerate(roles))
role2idx = dict((v, i) for i, v in enumerate(roles))
idx2skill = dict((i, v) for i, v in enumerate(skills))
skill2idx = dict((v, i) for i, v in enumerate(skills))

# 各データ長を取得
num_workers = len(workers)
num_locations = len(locations)
num_roles = len(roles)
num_assigns = len(assigns)
num_skills = len(skills)

### Amplify による定式化

それでは Amplify を用いて定式化します。最初に変数 $M$ を表す `role_variables` と変数 $P$ を表す `assign_variables` を `gen_symbols` 関数で作成します。二つの変数テーブルを構築するため、`gen_symbols` 関数の生成する変数のインデックスの開始番号を与えて重複しないようにします。

In [ ]:
# 従業員iが役職jで店舗lに勤務することを表す変数
gen = BinarySymbolGenerator()
role_variables = gen.array(num_workers, num_roles, num_locations)

# 従業員iが役職hで店舗lに勤務することを表す変数
assign_variables = gen.array(num_workers, num_assigns, num_locations)

これまでと同様に、勤務できない店舗や役職に関しては事前に変数の値を代入しておきます。

In [ ]:
for i, l in product(range(num_workers), locations):
    worker_req = df_worker_loc.iloc[i][l]
    if worker_req == 0:
        # 全ての役職で店舗割当が不可
        role_variables[i, :, loc2idx[l]] = 0
        # 全ての役割で店舗割当が不可
        assign_variables[i, :, loc2idx[l]] = 0

for i, j in product(range(num_workers), roles):
    worker_skill = df_worker_skill.iloc[i][j]
    if worker_skill == 0:
        # 全ての店舗で役職が不可
        role_variables[i, role2idx[j], :] = 0

for i in range(num_workers):
    if all(df_worker_skill.iloc[i][k] == 0 for k in skills):
        # 全ての店舗で役割(キッチン担当)が不可
        assign_variables[i, 1, :] = 0


# 勤務不可店舗・役職に関する変数が0に固定されている
print(role_variables)
# 担当不可役店舗・役割に関する変数が0に固定されている
print(assign_variables)

変数 $L$ を表す `location_variables` については、変数 $P$ から次のようにして得られます (変数 $M$ からも作成できます)。

In [ ]:
location_variables = assign_variables.sum(axis=1)

次に、各店舗、各調理スキルに対する充足率 $w$ を計算します。

$$
\begin{align*}
    w_{k,l} := \frac{1}{t_{k,l}}  \displaystyle \sum_{i\in W} s_{i,k} P_{i,1,l}
\end{align*}
$$

In [ ]:
# 充足率の計算
w = BinaryPolyArray(
    [
        sum_poly(
            df_worker_skill[idx2skill[k]] * assign_variables[:, 1, l],
        )
        / df_req[idx2skill[k]][l]
        for k in range(num_skills)
        for l in range(num_locations)
    ]
)

目的関数の各項目を計算します。最大化する目的についてはマイナスを付けることで最小化問題に変換します。

**充足率の平均の最大化**

$$
\begin{matrix}
    {\rm minimize} & -\left< w_{k,l} \right>
\end{matrix}
$$

**充足率の分散の最小化**

$$
\begin{matrix}
    {\rm minimize} & \left< w_{k,l}^2 \right> - \left< w_{k,l} \right>^2
\end{matrix}
$$

**従業員の希望度の最大化**

$$
\begin{matrix}
    {\rm minimize}& \displaystyle -\sum_{i\in W}\sum_{l\in S}c_{i,l}L_{i,l}
\end{matrix}
$$

In [ ]:
# 充足率の平均の最大化
average_fill_rate_cost = -((w.sum() / w.size) ** 2)

# 充足率の分散の最小化
variance_fill_rate_cost = (w * w).sum() / w.size - (w.sum() / w.size) ** 2

# 従業員の希望度最大化
location_cost = -sum_poly(
    num_workers,
    lambda i: sum_poly(
        num_locations,
        lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i, l],
    ),
)

最後に制約条件を与えます。

**従業員$i$は同時に1店舗のみ勤務する**

$$
\begin{align*}
    \forall i\in W, \sum_{l \in S}L_{i,l} = 1
\end{align*}
$$

**各店舗の要求人数に等しい管理職を配置する**

$$
    \forall l \in S, \forall j \in \left\{0, 1 \right\}, \sum_{i\in W}M_{i,j,l} = r_{j,l}
$$

**各店舗の要求人数以上のホール担当を配置する**

$$
\begin{align*}
    \forall l \in S, \sum_{i\in W} P_{i,0,l} = r_{l}
\end{align*}
$$

**ある店舗で役職を持つ(持たない)ならその店舗で役割を持つ(持たない)**

$$
    \forall i\in W, \forall l \in S, \sum_{j \in R} M_{i,j,l} = \sum_{h \in A} P_{i,h,l}
$$

全ての制約は等式制約なので `equal_to` を用います。

In [ ]:
# 従業員iは同時に1店舗のみ勤務できる
location_constarints = sum(one_hot(location_variables[i]) for i in range(num_workers))

# 各店舗の要求人数に等しい管理職を配置する
req_manager_constraints = sum(
    equal_to(role_variables[:, 0, l], df_req["manager"][l])
    for l in range(num_locations)
)
req_submanager_constraints = sum(
    equal_to(role_variables[:, 1, l], df_req["submanager"][l])
    for l in range(num_locations)
)

# 各店舗の要求人数以上のホール担当を配置する
req_hall_constraints = sum(
    equal_to(assign_variables[:, 0, l], df_req["hall"][l]) for l in range(num_locations)
)

# 変数Mと変数Pの関係
role_assign_constraints = sum(
    equal_to(
        (role_variables.sum(axis=1))[i, l] - (assign_variables.sum(axis=1))[i, l], 0
    )
    for i in range(num_workers)
    for l in range(num_locations)
)

目的関数と制約式から最適化モデルを作成します。これまでと同様に適切な重みを与えます。制約条件の重みは目的関数の取り得る値より大きめの値を推定して決定します。ここでは $10$ 程度の値を与えておけば十分です

In [ ]:
# それぞれの目的関数の係数
loc_priority = 1
ave_fill_priority = 1
var_fill_priority = 10

# 目的関数
cost_func = (
    loc_priority * location_cost
    + ave_fill_priority * average_fill_rate_cost
    + var_fill_priority * variance_fill_rate_cost
)

# 制約条件を表すペナルティ関数の重み
constraint_weight = 10

# 制約条件
constraints = constraint_weight * (
    location_constarints
    + req_manager_constraints
    + req_submanager_constraints
    + req_hall_constraints
    + role_assign_constraints
)

# 最適化モデル
model = cost_func + constraints

イジングマシンを実行します

In [ ]:
# 最適化の実行
result = solver.solve(model)

# 制約条件チェック
if len(result) == 0:
    raise RuntimeError("The given constraints are not satisfied")
values = result[0].values
energy = result[0].energy

`decode` メソッドを用いて、変数 `location_varibales`, `role_variables`, `assign_variables` の解として取り出します。取得した解をそれぞれ `location_solutions`, `role_solutions`, `assign_solutions` とします。

In [ ]:
# 割当店舗に関する変数の解
location_solutions = location_variables.decode(values)

# 割当店舗と役割に関する変数の解
role_solutions = role_variables.decode(values)

# 割当店舗と役職に関する変数の解
assign_solutions = assign_variables.decode(values)

### 結果の解析

結果から従業員がそれぞれがどこの店舗で勤務を行うのかを出力します。 変数 `role_solutions` において、`role_solutions[i][j][l] = 1` であれば、従業員 $i$ は役職 $j$ として店舗 $l$ で勤務することを表します。したがって、解が$1$であるインデックスを取り出すことで、どの従業員がどの役職、店舗で勤務するのかを取得できます。

In [ ]:
import numpy as np
from collections import defaultdict

(role_index_list, loc_index_list) = np.where(np.array(role_solutions) == 1)[1:]
dict_df = defaultdict(list)

for i, (j, l) in enumerate(zip(role_index_list, loc_index_list)):
    ## 配属勤務地
    worker_id = df_worker_loc.loc[i]["worker_id"]
    role = roles[j]
    loc = locations[l]
    dict_df["worker_id"].append(worker_id)
    dict_df["role"].append(role)
    dict_df["location"].append(loc)

df_result = pd.DataFrame.from_dict(dict_df, orient="index").T
display(df_result.style.hide_index())

次に、店舗ごとに要求スキル量がどの程度満たされているかを出力します。`hall`または`kitchen`のどちらに割り当てられたのかを`assign_solutions`の結果から取り出し、`kitchen`の場合には、その従業員が割り当てられている店舗の調理スキル量に従業員の調理スキル量それぞれを加算していきます。
表の各セルにおいて、**(合計スキル量)/(要求スキル量)**で算出した充足率 (fill rate) を出力しています。

In [ ]:
(assign_list, loc_index_list) = np.where(np.array(assign_solutions) == 1)[1:]

dict_result_loc = defaultdict(lambda: defaultdict(int))

for i, (j, l) in enumerate(zip(assign_list, loc_index_list)):
    # kitchenならば
    assign = assigns[j]
    worker_id = df_worker_loc.loc[i]["worker_id"]
    loc = locations[l]

    if assign == "kitchen":
        # すべての調理スキルの足し算を行う。
        for skill in skills:
            dict_result_loc[loc][skill] += df_worker_skill.loc[i][skill]

    else:
        dict_result_loc[loc]["hall"] += 1

df_result_loc = pd.DataFrame.from_dict(dict_result_loc, orient="index")

dict_result = defaultdict(defaultdict)

for i in range(len(df_result_loc)):
    loc = df_result_loc.iloc[i].name
    for skill in skills:
        require_num_skill = df_req[df_req["location"] == loc][skill].item()
        satisfy_num_skill = df_result_loc.iloc[i][skill].item()
        dict_result[loc][f"{skill}_fillrate"] = f"{satisfy_num_skill/require_num_skill}"

df_result = pd.DataFrame.from_dict(dict_result, orient="index")
display(df_result.style)

各店舗の要求スキル量が満たされていることがわかります。